In [34]:
import math
import torch
import numpy as np

import tensorflow as tf
from keras.layers import Input, AveragePooling2D

from keras.models import Model
from keras.applications.inception_v3 import InceptionV3

from keras.applications.inception_v3 import preprocess_input as incv3_preprocess_input

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.stats import multivariate_normal

import pandas as pd

import os

# Importing Data and feature extraction using pretrained Inception V3

In [2]:
input_shape = (299,299,3)

def create_model_incv3():
    tf_input = Input(shape=input_shape)
    model = InceptionV3(input_tensor=tf_input, weights='imagenet', include_top=False)
    output_pooled = AveragePooling2D((8, 8), strides=(8, 8))(model.output)
    return Model(model.input, output_pooled)

create_model = create_model_incv3

batch_of_images_placeholder = tf.TensorSpec(shape=(None, 32, 32, 3), dtype=tf.uint8)

batch_size = 16

tf_resize_op = lambda images: tf.image.resize(tf.cast(images, tf.float32), input_shape[:2])

preprocess_input = incv3_preprocess_input

# Define generator
def data_generator(data, labels=None):
    def generator():
        n = data.shape[0]
        start = 0
        while start < n:
            end = min(start + batch_size, n)
            batch_of_images = data[start:end]
            batch_of_images_resized = tf_resize_op(batch_of_images)
            batch_of_images_preprocessed = preprocess_input(batch_of_images_resized)
            if labels is not None:
                batch_of_labels = labels[start:end]
            else:
                # Create dummy labels as a placeholder when no labels are provided
                batch_of_labels = np.zeros((batch_of_images_preprocessed.shape[0],))
            yield (batch_of_images_preprocessed, batch_of_labels)
            start += batch_size
    return generator

model_extract = create_model()

In [3]:
def extract_features(I, model_extract, data_generator, batch_size):
    # Load the training and evaluation datasets using `I`
    t_train = torch.load(f'dataset/part_two_dataset/train_data/{I}_train_data.tar.pth')
    x_train = t_train['data']  # numpy.ndarray
    t_eval = torch.load(f'dataset/part_two_dataset/eval_data/{I}_eval_data.tar.pth')
    x_eval, y_eval = t_eval['data'], t_eval['targets']  # numpy.ndarray

    # Convert data to numpy arrays
    x_train = np.array(x_train)
    x_eval = np.array(x_eval)
    y_eval = np.array(y_eval).flatten()  # Flatten y_eval for consistency

    # Get number of samples in training and evaluation sets
    n_training = x_train.shape[0]
    n_evaling = y_eval.shape[0]

    # Generate data for model extraction
    data_train_gen = data_generator(x_train, None)()
    ftrs_training = model_extract.predict(data_train_gen, steps=math.ceil(n_training / batch_size), verbose=1)

    data_eval_gen = data_generator(x_eval, None)()
    ftrs_evaling = model_extract.predict(data_eval_gen, steps=math.ceil(n_evaling / batch_size), verbose=1)

    # Extract features by flattening predictions
    x_train_extracted = np.array([ftrs_training[i].flatten() for i in range(n_training)])
    x_eval_extracted = np.array([ftrs_evaling[i].flatten() for i in range(n_evaling)])

    return x_train_extracted, x_eval_extracted, y_eval

for I in range(1,11):
    globals()[f"x_train_extracted_{I+10}"], globals()[f"x_eval_extracted_{I+10}"], globals()[f"y_eval_{I+10}"] = extract_features(
        I=I,
        model_extract=model_extract,
        data_generator=data_generator,
        batch_size=batch_size
    )

/tmp/ipykernel_7932/2421634456.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  t_train = torch.load(f'dataset/part_two_dataset/train_data/{I}_train_data.tar.pth')
/tmp/i

157/157 ━━━━━━━━━━━━━━━━━━━━ 124s 779ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 117s 749ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 124s 788ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 123s 782ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 124s 790ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 133s 845ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 123s 784ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 122s 777ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 128s 814ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 130s 829ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 123s 782ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 135s 858ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 148s 943ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 130s 826ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 129s 822ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 136s 866ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 136s 867ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 132s 842ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 131s 835ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 130s 830ms/step


# Saving extracted features and labels

In [38]:
for i in range(11, 21):
    feature = eval(f"x_train_extracted_{i}")
    folder_path = "extracted_features"
    os.makedirs(folder_path, exist_ok=True)
    file_name = f"x_train_extracted_{i}.npy"
    file_path = os.path.join(folder_path, file_name)
    np.save(file_path, feature)

for i in range(11, 21):
    feature = eval(f"x_eval_extracted_{i}")
    folder_path = "extracted_features"
    os.makedirs(folder_path, exist_ok=True)
    file_name = f"x_eval_extracted_{i}.npy"
    file_path = os.path.join(folder_path, file_name)
    np.save(file_path, feature)

for i in range(11, 21):
    feature = eval(f"y_eval_{i}")
    folder_path = "extracted_features"
    os.makedirs(folder_path, exist_ok=True)
    file_name = f"y_eval_{i}.npy"
    file_path = os.path.join(folder_path, file_name)
    np.save(file_path, feature)

# Loading Saved Extracted features from current task and Evaluation features from Previous Task

In [39]:
for i in range(1,21):
    globals()[f'x_eval_extracted_{i}'] = np.load(f'extracted_features/x_eval_extracted_{i}.npy')
    globals()[f'y_eval_{i}'] = np.load(f'extracted_features/y_eval_{i}.npy')

for i in range(11,21):
    globals()[f'x_train_extracted_{i}'] = np.load(f'extracted_features/x_train_extracted_{i}.npy')

# Generative Classifier Continual Model

In [40]:
class GenerativeClassifierContinual:
    def __init__(self, n_classes, n_features, alpha=0.5):
        self.n_classes = n_classes
        self.n_features = n_features
        self.means = np.zeros((n_classes, n_features))
        self.covariances = np.array([np.eye(n_features) for _ in range(n_classes)])
        self.alpha = alpha  # Regularization weight

    def fit(self, X_train, y_train, prev_means=None, prev_covariances=None):
        new_means = np.zeros((self.n_classes, self.n_features))
        new_covariances = np.array([np.eye(self.n_features) for _ in range(self.n_classes)])

        # Calculate Gaussian parameters for current data
        for c in range(self.n_classes):
            class_samples = X_train[y_train == c]
            if len(class_samples) > 0:
                new_means[c] = np.mean(class_samples, axis=0)
                if len(class_samples) > 1:
                    new_covariances[c] = np.cov(class_samples, rowvar=False) + 1e-6 * np.eye(self.n_features)

        # Regularize means and covariances with the previous model's parameters
        if prev_means is not None and prev_covariances is not None:
            self.means = self.alpha * prev_means + (1 - self.alpha) * new_means
            self.covariances = self.alpha * prev_covariances + (1 - self.alpha) * new_covariances
        else:
            self.means = new_means
            self.covariances = new_covariances

    def generate_replay_data(self, num_samples_per_class=50):
        # Generate synthetic data based on current Gaussian parameters
        replay_data = []
        replay_labels = []
        for c in range(self.n_classes):
            samples = np.random.multivariate_normal(self.means[c], self.covariances[c], num_samples_per_class)
            replay_data.append(samples)
            replay_labels.append(np.full(num_samples_per_class, c))
        return np.vstack(replay_data), np.concatenate(replay_labels)

    def predict(self, X):
        # Calculate log probabilities for each class
        log_probs = np.zeros((X.shape[0], self.n_classes))
        for c in range(self.n_classes):
            distribution = multivariate_normal(mean=self.means[c], cov=self.covariances[c])
            log_probs[:, c] = distribution.logpdf(X)

        # Assign each sample to the class with the highest log probability
        predictions = np.argmax(log_probs, axis=1)
        return predictions

    def score(self, X, y):
        predictions = self.predict(X)
        accuracy = np.mean(predictions == y)
        return accuracy

# Loading the models and transformation saved from previous task

In [41]:
import joblib  # Or use pickle if saved with it

model_list = joblib.load('model_list.joblib')
scaler = joblib.load('scaler.joblib')
pca = joblib.load('pca.joblib')

# Model Training

In [42]:
n_classes=10
pca_components=40

# Continual learning with Gaussian class-conditionals, prototype regularization, and generative replay
for i in range(11, 21):
    # Standardize and apply PCA to the current dataset
    x_train_scaled = scaler.transform(globals()[f'x_train_extracted_{i}'])
    x_train_pca = pca.transform(x_train_scaled)

    # Generate replay data using the previous model
    replay_data, replay_labels = model_list[-1].generate_replay_data(num_samples_per_class=50)

    # Combine replay data with current data
    x_combined = np.vstack([x_train_pca, replay_data])
    y_combined = np.concatenate([model_list[-1].predict(x_train_pca), replay_labels])

    # Train a new model with regularized Gaussian parameters
    model = GenerativeClassifierContinual(n_classes, n_features=pca_components, alpha=0.7)
    model.fit(x_combined, y_combined,
              prev_means=model_list[-1].means, prev_covariances=model_list[-1].covariances)

    # Append the new model to the model list
    model_list.append(model)


# Accuracy checking of different Models with different datasets

In [43]:
# Evaluate each model on the held-out datasets
accuracy_matrix = np.zeros((10, 20))
for i, model in enumerate(model_list[10:], start=11):
    for j in range(1, 21):
        x_eval_scaled = scaler.transform(globals()[f'x_eval_extracted_{j}'])
        x_eval_pca = pca.transform(x_eval_scaled)
        eval_labels = globals()[f'y_eval_{j}']
        accuracy_matrix[i-11, j-1] = model.score(x_eval_pca, eval_labels) * 100

print("Accuracy Matrix:")

accuracy_matrix = pd.DataFrame(
    accuracy_matrix,
    index=[f"Model {i}" for i in range(11,21)],
    columns=[f"Eval Dataset {j+1}" for j in range(20)]
)

print(accuracy_matrix)

Accuracy Matrix:
          Eval Dataset 1  Eval Dataset 2  Eval Dataset 3  Eval Dataset 4  \
Model 11           78.20           78.04           76.80           77.36   
Model 12           77.36           77.56           76.92           77.04   
Model 13           76.44           77.80           75.80           76.24   
Model 14           76.52           77.44           75.68           76.08   
Model 15           76.52           76.88           75.48           76.08   
Model 16           75.72           76.48           74.72           75.56   
Model 17           75.76           76.40           74.72           75.36   
Model 18           75.00           76.08           74.64           74.56   
Model 19           74.84           76.16           74.24           74.12   
Model 20           74.44           76.00           74.08           73.76   

          Eval Dataset 5  Eval Dataset 6  Eval Dataset 7  Eval Dataset 8  \
Model 11           77.44           77.80           77.12           78.